In [1]:
import os
import sys
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader


In [2]:
from dotenv import load_dotenv 
from pathlib import Path
load_dotenv()

project_root = Path(os.path.abspath('')).parent
sys.path.append(str(project_root))

In [3]:
INTERMEDIATE_DATA_DIR = os.getenv("INTERMEDIATE_DATA_DIR")
NIH_CXR14_DATASET_DIR = os.getenv("NIH_CXR14_DATASET_DIR") + "/original"
print(NIH_CXR14_DATASET_DIR)
print(INTERMEDIATE_DATA_DIR)

/home/yasin/Lfstorage/datasets/nih-cxr14/original
./data


In [4]:
from src.utils import VaeFeatureExtractor
from src.datasets import NIHImageDataset
from src.pipelines import VaeProcessor


/home/yasin/Lfstorage/Projects/cxr-diffusion/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-03-10 04:08:24.518877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741568904.541563 1521762 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741568904.548443 1521762 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 04:08:24.575010: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.


In [5]:
device = "cuda:0"
batch_size = 16
num_workers = 24
size = 512
save_dir = INTERMEDIATE_DATA_DIR + f"/{size}_vae_features"

In [15]:
vae_processor = VaeProcessor(device=device)

def transform(image):
    output = vae_processor.image_processor.preprocess(image=image, width=size, height=size)
    output = output.squeeze(0)
    return output
dataset = NIHImageDataset(NIH_CXR14_DATASET_DIR , transform=transform)


dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

In [16]:
sample = next(iter(dataloader))
sample[1].shape


torch.Size([16, 3, 512, 512])

In [ ]:
from tqdm.auto import tqdm

for i, batch in enumerate(tqdm(dataloader)):
    print(i)
    print(type(batch))
    print(len(batch))
    print(type(batch[0]))
    print(len(batch[0]))
    break


In [ ]:

feature_extractor = VaeFeatureExtractor(
    processor=vae_processor,
    dataloader=dataloader,
    save_dir=save_dir,
    save_name="feature",
    chunk_size=100
)   



In [ ]:
feature_extractor.run()